In [2]:
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

AMPL Development Version 20240404 (MSVC 19.38.33135.0, 64-bit)
Demo license with maintenance expiring 20260131.
Using license file "c:\Users\thuduong\Anaconda3\envs\optima\Lib\site-packages\ampl_module_base\bin\ampl.lic".



### purpose: testing optima with generate all pattern but the max of each FG

In [8]:
## PARAMETER - CASE 0
PARAMS = {"warehouse": "HSC"
          ,"spec_name": "JSH270C-PO" # yeu cau chuan hoa du lieu OP - PO
          ,"thickness": 1.6
          ,"maker" : "CSVC"
          ,"stock_ratio": { #get from app 
                    "limited": None
                    # "default": 2
                    # "user_setting": 4
                }
        #   ,"forecast_scenario": median
          }

margin_dict = { "HSC":
    {#thickness : min_margin
    2.6:10
    ,2 : 8
   , 1.6: 6
    },
    "NQS":
    {#save margin_dict in azure env, margin dictionary by warehouse, margin by range?
    2.6:10
    ,2 : 8
   , 1.6: 6
    }
}
# GET ALL PARAMS
MIN_MARGIN = margin_dict[PARAMS["warehouse"]][PARAMS["thickness"]]
print(f"MIN_MARGIN:{MIN_MARGIN}")

BOUND_KEY = next(iter(PARAMS['stock_ratio']))
BOUND_VALUE = PARAMS['stock_ratio'][BOUND_KEY]
print(f"BOUND_VALUE:{BOUND_VALUE}")

In [11]:
from codes.process_df import filter_by_params
import pandas as pd
import numpy as np

# STEP 1: FILTER DF BY PARAMS
stock_file_path = "data/test_mc_df.xlsx"
finish_file_path = 'data/test_finish_df_Jul2022.xlsx'
stock_key, value_columns = "inventory_id", ['receiving_date','status',"width", "weight"]
finish_df = filter_by_params(finish_file_path, PARAMS)
stock_df = filter_by_params(stock_file_path, PARAMS)

In [12]:
# STEP 2: PROCESS DATA TO DF THAT CAN USE TO CONVERT TO DICT
filter_stock_df


In [13]:
#STEP 3: SET VALUE AND PARAMS TO AMPL
# 1. Send the data from "amt_df" to AMPL and initialize the indexing set "FOOD"
ampl.set_data(filter_stock_df, "FOOD")
# 2. Send the data from "nutr_df" to AMPL and initialize the indexing set "NUTR"
ampl.set_data(nutr_df, "NUTR")
# 3. Set the values for the parameter "amt" using "amt_df"
ampl.get_parameter("amt").set_values(amt_df)

In [4]:
%%ampl_eval
reset;
set F;
set P;
# width stock
param width_s integer;
# width stock minus min_margin
param width_s_min_margin integer;
# weight per unit of stock
param wu > 0;
# width finished pieces
param width_f{F}
# upper bound with over-cut
param f_upper_demand{F};
param demand_finish{F};
# how many f pieces are returned from pattern p
param a{F, P};
# which stock s is choosen for pattern p
var b{P} binary;
# Find the patterns of stock that minimize the loss
minimize trim_loss:
  width_s - sum{p in P, f in F} b[p] * a[f,p] * width_f[f];

subject to assign_each_finish_to_pattern:
  sum{p in P} b[p]  = 1;

subject to feasible_pattern_max_margin:
  sum{p in P, f in F} b[p]* a[f,p] * width_f[f] >= 0.96 * width_s;

subject to feasible_pattern_min_margin:
  sum{p in P, f in F}  b[p] * a[f,p] * width_f[f] <= width_s_min_margin;
subject to weight_demand {f in F}:
  sum{p in P} b[p] * a[f,p] * width_f[f] * wu <= f_upper_demand[f];

In [10]:
FOOD = ampl.get_data("FOOD")
FOOD.toPandas()

""
BEEF
CHK
FISH
HAM
MCH
MTL
SPG
TUR


In [ ]:
%%ampl_eval
param nPatterns integer > 0;
set PATTERNS = 1..nPatterns; # patterns
set WIDTHS; # finished widths
param order {WIDTHS} >= 0; # rolls of width j ordered
param overrun; # permitted overrun on any width
param rolls {WIDTHS,PATTERNS} >= 0 default 0; # rolls of width i in pattern j

var Cut {PATTERNS} integer >= 0; # raw rolls to cut in each pattern

minimize TotalRawRolls: sum {p in PATTERNS} Cut[p];

subject to FinishedRollLimits {w in WIDTHS}:
  order[w] <= sum {p in PATTERNS} rolls[w,p] * Cut[p] <= order[w] + overrun;

In [ ]:
  set WIDTHS;
    param W >= 0;
    param v{WIDTHS} >= 0;
    var x{WIDTHS} integer >= 0;
    maximize profit: sum {w in WIDTHS} v[w]*x[w];
    subject to capacity: sum {w in WIDTHS} w*x[w] <= W;